In [26]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import math

#Load Import File
data_fp = str(("/Users/bkanchipurambaburavi/Downloads/IOT4.xlsx"))
data_file = pd.ExcelFile(data_fp)
t_forecast = pd.read_excel(data_file, sheet_name='T Forecast')
t_forecast = t_forecast.drop(axis=1,columns="Update Date")
#Changing the datetime format for time series data to short date format
for i in range(13,115):
    t_forecast = t_forecast.rename(columns={t_forecast.columns[i]: t_forecast.columns[i].strftime("%m/%d/%Y")})
#Reading b_h and comp sheet
b_h = pd.read_excel(data_file, sheet_name='Harnesses')
b_h = b_h.drop(columns='Coax?')
comp = pd.read_excel(data_file, sheet_name='Components')

print("Ok")

Ok


In [27]:
#Clean b_h sheet
b_h_col_rename = {'Harness/TLA PN':'Harness/PCBA PN'}
b_h = b_h.rename(columns=b_h_col_rename)

#Clean Components Sheet, nothing to put here yet
comp_col_rename = {'T PN':'TPN', 'Harness PN':'Harness/PCBA PN', 'BOM UOM':'UOM'}
comp = comp.rename(columns=comp_col_rename)

#manipulating the split date format of date intervals to a specific entity for comparison with status change date

for i in range(len(b_h['Status Change Date'])):
    if (b_h.iloc[i,14] == str('10/24/2022 - 11/1/2023')) is True:
        b_h.iloc[i,14] = str('2015-01-01 00:00:00')
    else:
        b_h.iloc[i,14] = b_h.iloc[i,14]    
        
for i in range(len(b_h["Status Change Date"])):
    if pd.isna(b_h.iloc[i,14]) is True:
        b_h.iloc[i,14] = pd.to_datetime(str('2013-01-01 00:00:00'))
    else:
        b_h['Status Change Date'] = pd.to_datetime(b_h['Status Change Date'])
        
 #Changing the datetime format for time series data to short date format
b_h['Status Change Date'] = pd.to_datetime(b_h['Status Change Date'])
b_h['Status Change Date'] = b_h['Status Change Date'].dt.strftime('%m/%d/%Y')

print(b_h['Status Change Date'])

0      01/01/2013
1      01/01/2013
2      01/01/2013
3      01/01/2013
4      01/01/2013
          ...    
652    10/30/2023
653    10/30/2023
654    10/30/2023
655    10/30/2023
656    10/30/2023
Name: Status Change Date, Length: 657, dtype: object


In [28]:
#Merge T Forecast with b_h
b_h = pd.merge(b_h,t_forecast,how='left')
print(b_h.shape)
print(b_h.head())
print(b_h.tail())

#Parse forecast qty columns
print(list(b_h.columns))
num_col_start = str("Offset")
num_col_start_indx = b_h.columns.get_loc(num_col_start) + 1
b_h_qtys = b_h.iloc[:, num_col_start_indx:]
print(b_h_qtys.head())

#Multiply forecast qty by QPV
b_h['QPV'] = pd.to_numeric(b_h['QPV'], downcast='float')
b_h_forecast = b_h_qtys.mul(b_h['QPV'],axis=0)
print(b_h_forecast.head())

#Update board and harness forecast numbers to reflect multiplied QPV
b_h.iloc[:, num_col_start_indx:] = b_h_forecast
print(b_h.head())

#Offset Updation
b_h.iloc[:,27:] = b_h.apply(lambda x: x.iloc[27:].shift(-x.loc['Offset']),axis=1,result_type='expand')
b_h = b_h.fillna(value=None, method='ffill', axis=1, inplace=False, limit=None, downcast=None)
print(b_h.head())
#print(b_h.shape)
print("Done")

print(b_h.head())
print(list(b_h.columns))

#Based on comparing the status change date and time series date, multipying the respective by current or future QPV

for column in range(28,130):
    for row in range(len(b_h['Status Change Date'])):
        if (datetime.strptime(b_h['Status Change Date'][row], '%m/%d/%Y') == pd.to_datetime("01/01/2015")) is True:
            if datetime.strptime(b_h.columns[column], '%m/%d/%Y') < datetime.strptime(str("10/24/2022"), '%m/%d/%Y'):
                b_h.iloc[row,column] = 0
            if datetime.strptime(b_h.columns[column], '%m/%d/%Y') > datetime.strptime(str("11/1/2023"), '%m/%d/%Y'): 
                b_h.iloc[row,column] = 0
            else: 
                b_h.iloc[row,column] = b_h.iloc[row,column]*b_h.iloc[row,11]
        if (datetime.strptime(b_h['Status Change Date'][row], '%m/%d/%Y') == pd.to_datetime("01/01/2013")) is True:
            b_h.iloc[row,column] = b_h.iloc[row,column]*b_h.iloc[row,11]
        
        else:
            if (datetime.strptime(b_h['Status Change Date'][row], '%m/%d/%Y') > datetime.strptime(str(b_h.columns[column]), '%m/%d/%Y')) is True:
                b_h.iloc[row,column] = b_h.iloc[row,column]*b_h.iloc[row,10]
            else:
                b_h.iloc[row,column] = b_h.iloc[row,column]*b_h.iloc[row,11]
            
print('ok')

for i in range(len(b_h['Status Change Date'])):  
    if (datetime.strptime(b_h['Status Change Date'][i], '%m/%d/%Y') == pd.to_datetime("01/01/2013")) is True:
        b_h.iloc[i,14] = str('') 
    elif (datetime.strptime(b_h['Status Change Date'][i], '%m/%d/%Y') == pd.to_datetime("01/01/2015")) is True:
        b_h.iloc[i,14] = str('10/24/2022 - 11/1/2023')
    else:
        b_h.iloc[i,14] = b_h.iloc[i,14]

print('ok')


#rounding
#converting float to integer
for i in range(27,130):
    for j in range(len(b_h)):
        b_h.iloc[j,i] = int(b_h.iloc[j,i])

        
#pivot summary after offset updation
for i in range (0,4): 
    b_h.iloc[:, 16+i] = b_h.iloc[:,28 + 13*i : 28 + 13*(i+1)].sum(axis = 1)

for i in range (0,4): 
    b_h.iloc[:, 18+i] = b_h.iloc[:,41 + 13*i : 41 + 13*(i+1)].sum(axis = 1)
    
for i in range (0,4): 
    b_h.iloc[:, 23+i] = b_h.iloc[:,93 + 13*i : 93 + 13*(i+1)].sum(axis = 1)
    
#quarterly and yearly pivot summary    
b_h.iloc[:, 17] = b_h.iloc[:, 16:17].sum(axis = 1) 
b_h.iloc[:, 22] = b_h.iloc[:, 18:22].sum(axis = 1)
b_h.iloc[:, 27] = b_h.iloc[:, 23:27].sum(axis = 1)


print(b_h)
print("Done")        
        
        

In [29]:
#Change b_h sheet column names for comp sheet
b_h_rename = {'Description':'Harness/PCBA Description'}
b_h_comp = b_h.rename(columns=b_h_rename)

#rename status to status update as both the sheets have the same column name
b_h_comp = b_h_comp.rename(columns={'Status': 'Status Update'})

#drop the Current QPV, Future QPV, QPV, Ship to Location columns
b_h_comp = b_h_comp.drop(columns=['Current QPV','Future QPV','QPV','Ship to Location'])
#print(b_h_comp.head())

#Load forecast into harnesses sheet
#print(comp.columns)
#print(b_h_comp.columns)
comp = pd.merge(comp,b_h_comp,how='left')
#print(comp.shape)
print(comp.head())

#Parse forecast qty columns
print(list(comp.columns))
comp_num_col_start = str('Offset')
comp_num_col_start_indx = comp.columns.get_loc(comp_num_col_start) + 1
comp_qtys = comp.iloc[:, comp_num_col_start_indx:]
print(comp_qtys.head())

#Multiply forecast qty by QPV
#comp['Qty'] = pd.to_numeric(comp['Qty'], downcast='float')
comp_forecast = comp_qtys.mul(comp['Qty'],axis=0)
#print(comp_forecast.head())

#Update board and harness forecast numbers to reflect multiplied QPV
comp.iloc[:, comp_num_col_start_indx:] = comp_forecast

#drop the status column
comp = comp.drop(columns=['Status'])
print(comp.head())
#print(comp.shape)
print("Done")


In [30]:
#Write dataframes to Excel file and save to local drive
fp = str(input("Please enter output file name:\n"))

#generating separate component level forecast sheet for each suppliers
with pd.ExcelWriter(fp + '.xlsx') as writer:
    t_forecast.to_excel(writer, index=False, sheet_name='High Level Forecast')
    b_h.to_excel(writer, index=False, sheet_name='Harnesses Forecast')
    comp.to_excel(writer, index=False, sheet_name='Components Forecast')
    for i in range(0,n):
        comp[comp.Manufacturer == str(manufacturer[i])].to_excel(writer, index=False, sheet_name=str(manufacturer[i]))
        
#Data Summary for each suppliers
#fixing supplier names
comp[['Manufacturer','Remove']] = comp['Manufacturer'].str.split(' ', n=1, expand=True)
comp = comp.replace('A','T')

print(list(comp.columns))

print(comp['Manufacturer'].unique())

comp = comp.query("Manufacturer == ['R']")
print(comp)
manufacturer = comp['Manufacturer'].unique()
n = len(manufacturer)
n
        
        
#drop offset column for supplier files
comp = comp.drop(columns=['Offset'])

for i in range(0,n):
    with pd.ExcelWriter(fp + str(manufacturer[i]) + '.xlsx') as writer:
        comp[comp.Manufacturer == str(manufacturer[i])].to_excel(writer, index=False, sheet_name=str(manufacturer[i]))
        
    
print('Saved')